## Import the required libraries

In [1]:
#pip install selenium
import pandas as pd #data analysis

from bs4 import BeautifulSoup #web scraper/data parser

#Browser automation
import selenium.webdriver 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys

import time
from datetime import datetime, timedelta

## We decide the timestamp between which we want the data

In [2]:
start_date = datetime.strptime("28/08/2020", "%d/%m/%Y")
end_date = datetime.strptime("30/08/2020", "%d/%m/%Y")
delta = timedelta(days = 1)
dates = []
while start_date <= end_date:
    dates.append(start_date.strftime("%d/%m/%Y"))
    start_date = start_date + delta

In [3]:
dates

['28/08/2020', '29/08/2020', '30/08/2020']

## Scrape function that takes two parameters: source & destination

In [4]:
#Defining the web crawler fucntion
def scrape(source, destination):
    
    data = pd.DataFrame([])
    trDate = dates[0]
    origin = source
    destin = destination
    baseDataUrl = "https://www.makemytrip.com/flight/search?itinerary="+ origin +"-"+ destin +"-"+ trDate +"&tripType=O&paxType=A-1_C-0_I-0&intl=false&cabinClass=E"
    driver = selenium.webdriver.Chrome(r"C:\Users\deepu\Downloads\chromedriver.exe") #Change URL accordingly
    driver.get(baseDataUrl) #Opening the base data URL
    time.sleep(5) #Waiting 5 seconds for webpage to load

    for i in range(0, len(dates)):
        
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(1)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        body = driver.find_element_by_tag_name("body").get_attribute("innerHTML")
        #driver.quit()

        soupBody = BeautifulSoup(body) # Parse the inner HTML using BeautifulSoup

        spanFlightName = soupBody.findAll("span", {"class": "airways-name"}) # Tags with Flight Name
        divDeptTime = soupBody.findAll("div", {"class": "dept-time"}) # Tags with Departure Time
        pDeptCity = soupBody.findAll("p", {"class": "dept-city"}) # Tags with Departure City
        pFlightDuration = soupBody.findAll("p", {"class": "fli-duration"}) # Tags with Flight Duration
        pArrivalTime = soupBody.findAll("p", {"class": "reaching-time append_bottom3"}) # Tags with Arrival Time
        pArrivalCity = soupBody.findAll("p", {"class": "arrival-city"}) # Tags with Arrival City
        spanFlightCost = soupBody.findAll("span", {"class": "actual-price"}) # Tags with Flight Cost
        pTotalStops = soupBody.findAll("p", {"class": "fli-stops-desc"}) # Tags with Total Stops
        
        flightsData = [["flight_name", "departure_time", "departure_city", "flight_duration", "arrival_time", "arrival_city", "flight_cost", "total_stops", 'date_of_journey', 'timestamp']]

        # Extracting data from tags and appending to main database flightsData
        for j in range(0, len(spanFlightName)):
            flightsData.append([spanFlightName[j].text, divDeptTime[j].text, pDeptCity[j].text, pFlightDuration[j].text, pArrivalTime[j].text, pArrivalCity[j].text, spanFlightCost[j].text, pTotalStops[j].text, dates[i], datetime.now().strftime("%d/%m/%Y")])

        
        data = data.append(pd.DataFrame(flightsData))
        print(dates[i])
        
        driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME) #scroll up
        time.sleep(2)
        
        try:
            driver.find_elements_by_xpath('//div[@class = "item "]')[i].click()
            time.sleep(5)
        except:
            driver.refresh()
        #data.columns = data.iloc[0]
        #data = data[1:]

    return data

In [5]:
df = scrape('BOM','DEL')

28/08/2020
29/08/2020
30/08/2020


In [6]:
df

,0,1,2,3,4,5,6,7,8,9
0,flight_name,departure_time,departure_city,flight_duration,arrival_time,arrival_city,flight_cost,total_stops,date_of_journey,timestamp
0,flight_name,departure_time,departure_city,flight_duration,arrival_time,arrival_city,flight_cost,total_stops,date_of_journey,timestamp
0,flight_name,departure_time,departure_city,flight_duration,arrival_time,arrival_city,flight_cost,total_stops,date_of_journey,timestamp


In [7]:
df.to_csv("C:/Users/deepu/Desktop/Python codes/Capstone Project/Del_Hyd_new.csv")

In [8]:
import os
import glob
import pandas as pd
os.chdir("C:/Users/deepu/Desktop/Python codes/Capstone Project")
all_filenames = [i for i in glob.glob('*.{}'.format('csv'))]
df = pd.concat([pd.read_csv(f) for f in all_filenames])
df=df.drop(['Unnamed: 0'],1)
df.columns = df.iloc[0]
df = df[df['flight_name'] !='flight_name']
df.rename(columns = {'datee':'Date_of_Journey', 'timee':'Day_of_Booking'}, inplace = True)
df.to_csv("C:/Users/deepu/Desktop/Python codes/Capstone Project/final_data.csv", index=False)

C:\Users\deepu\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  
